# Lab 4: Parameter spaces and optimization

__INTRODUCTION__: In this lab, we will look at airfoil shape optimisation using the simulator introduced in Lab 3 and the geometry tools that we have developed in Labs 1 and 2. We will use parametric airfoil geometries and then investigate how the airfoil performance depends on the geometry parameters. We will also use optimisation tools to find optimal geometries more rigorously than by trial and error.

Make sure you have watched the videos for Lecture 4 and the associated python demo video.

Similar to previous labs, there is a number of tasks for you to work on. The tasks are accompanied by a textual description to guide you. Often some code has been prepared for you in advance and you typically have to execute code cells (using `Shift+Enter` on your keyboard). 

In some cells, some code has been omitted and this is clearly highlighted by special sections like

```
   # YOUR CODE HERE
   
   # END YOUR CODE HERE  
```

Feel free to post a message on the discussion forum should you have any questions!

We first import the basic python modules for array operations and plotting, as usual.

In [1]:
import numpy as np
import matplotlib.pyplot as pl

# you can replace inline with qt to get plots in a separate window
%matplotlib inline

---

## Task 1 - Create a function that returns the drag coefficient for a given airfoil geometry

We will need some of the tools developed in the previous labs. As usual you should download the files `lab_1_tools.py`, `lab_2_tools.py` and `airfoilsim.py` from the BlackBoard folder for Lab 4 and put them in the same directory as this notebook you are using. __Note you'll have to be connected to the VPN!__

In [2]:
# for the geometry
from lab_2_tools import parametric_aerofoil, plot_airfoil, bezier_spline_airfoil

# for the simulation
from airfoilsim import airfoilsim

OK, for the first task will use the function `parametric_aerofoil` developed in Lab 2. Let's recall how it works by looking at its documentation:

In [3]:
?parametric_aerofoil

With this function we can obtain a one-parameter family of airfoil, parametrized by one variable, the weight $w_{\mathbf{q}_2}$ of point $\mathbf{q}_2$. Since the drag depends on the geometry, we expect that the sectional coefficient $C_d$ will depend on $w_{\mathbf{q}_2}$. Before we do anything else let's define some inflow conditions

In [4]:
# inflow parameters
V  = 15        # m/s
c  = 0.5       # m
nu = 1.516e-5  # m^2/s at 20 degrees Celsius (see https://www.engineersedge.com/physics/viscosity_of_air_dynamic_and_kinematic_14483.htm)

# Reynolds number
Re = V*c/nu
print(Re)
# angle of attack in degree
alpha = 2

494722.9551451187


I have selected some representative conditions for a low-speed aerodynamic flow, using the same properties as in Lab 3. 

To see how the drag coefficient depends on the weight $w_{\mathbf{q}_2}$ we can define a function, called simply `drag` that:

- takes the weight as a single input (with argume we call `w` for simplicity)
- creates the geometry
- runs the simulation and discards the pressure distribution
- returns the drag coefficient

I have made a start in the cell below and added some documentation and comments to help you constructing the function. As discussed in the demo lecture number 4 (see Blackboard), optimisation routines in the scipy library expect that the objective function takes one single array argument, i.e. the parameter vector. In this case, we have a single parameter, which we wrap in a one-element array. Inside the function, we need to make sure that we extract the only component of this array. 

In [5]:
def drag(w):
    """
    Return the drag coefficient for the 1D parametric airfoil defined in Lab 2 with
    weight for control point $q_2$ given by `w[0]`. Note that the input `w` should be 
    a one element list or a one element array, not a single scalar.
    """
    # YOUR CODE HERE
    # obtain geometry using `parametric_aerofoil` (one line of code)
    B, p, q = parametric_aerofoil(w[0])
    #print("This is B",B)
    #print("This is p",p)
    #print("This is q", q)

    
    # run flow calculations at alpha and Re defined in one of the previous cells (one line of code)
    cl, cd, x_lower, x_upper, cp_lower, cp_upper = airfoilsim(B, alpha, Re)
    
    # END YOUR CODE here
    return cd

### Testing

If your function has been implemented properly (and if you have not changed inflow conditions), the cell below should execute successfully.

In [6]:
# run this cell
assert drag([1]) == 0.0063173822

Note that we have to pass `[1]` as an argument and not `1`, to run calculations with $w_{\mathbf{q}_2} = 1$, because we have to pass an array or list of parameters. Try executing `drag(1)` in the cell below and make sure you understand the nature of the resulting error.

In [7]:
drag(1) # this produces an error

TypeError: 'int' object is not subscriptable

---

## Task 2 - Drag coefficient optimisation

Now that we have the drag function, we can try to find the value of the parameter $w_{\mathbf{q}_2}$ that minimises the drag. If the airfoil drag is lower, we burn less fuel and everyone (except those sitting on oil) likes that! 

Mathematically, we want to solve

\begin{equation}
    \begin{aligned}
        &\underset{  w_{\mathbf{q}_2} }{\text{min}}&& C_d{(w_{\mathbf{q}_2})}
    \end{aligned}
\end{equation}

where $C_d{(w_{\mathbf{q}_2})}$ indicates that the drag coefficient is a function of the weight $w_{\mathbf{q}_2}$ and we want to find the value of this weight that produces the minimum drag. Note that this is an unconstrained optimisation problem over one variable.

We will explore in detail the parameter space of this problem later on, but we'll first used an optimiser.


### Optimisation

We will use the `minimize` function from the subpackage `optimize` in the `scipy` library, let's import it first.

In [12]:
from scipy.optimize import minimize

As shown in the demo lecture 4, we will use the method `Nelder-Mead`. This method is a bit more robust than gradient based methods and you can read more [here](https://en.wikipedia.org/wiki/Nelder–Mead_method) and [here](http://www.scholarpedia.org/article/Nelder-Mead_algorithm) about this method if you want. We also need to specify a stopping tolerance, by passing a value to the keyword argument `tol`. A tolerance of four decimal places should be sufficient. 

We also need to provide a reasonable initial guess. Let's try to run the optimiser by executing the cell below.

In [13]:
# define initial guess, as a one-element list. A value of one for this weight could be a reasonable guess.
w_guess = [1.0]

# run the optimizer (this might take a while), by passing the function we want to minimize and a initial guess
out = minimize(drag, w_guess, tol=1e-4, method="Nelder-Mead")

I have stored the output of the `minimize` function in a dummy variable `out`. As we discussed in the demo lecture number 4, the optimal parameter vector (the minimizer) can be accessed with `out.x` (the `x` field of `out`), while the objective function at this optimal value is `out.fun`. Let's extract the minimiser first

In [14]:
# execute this cell with Shift+Enter to print out.x
out.x

array([1.0546875])

OK, the optimial parameter value is close to `1`, maybe a bit higher. The minimum drag coefficient is 

In [15]:
# execute this cell with Shift+Enter
out.fun

0.00629907

Not bad, but, how do we know we did find the minimum?

## Visualising how drag depends on the geometry

Now, by simply running the optimiser we can't see what's going on in parameter space and understand the role of this design parameter on the airfoil performance.


Hence, instead of running the optimiser we could simply explore the parameter space, by calculating the drag coefficient for several values of the weight $w_{\mathbf{q}_2}$. This is of course less efficient than using an optimiser, especially when there are more parameters, because we need to finely grid the parameter space and run lots of simulations. The optimiser, instead, only samples the objective function at few points, those necessary to descend to a minimum.

For this case, we can still attempt to map the parameter space, and this is the objective of the next task. You will have to write a function `parameter_sweep` that computes the drag coefficient of the parametric airfoil at for several values of the weight $w_{\mathbf{q}_2}$. I have added some documentation and comments to the function to help you in this task.

In [16]:
def parameter_sweep(alpha, Re, Nw = 30):
    """ 
    Compute the drag coefficient of the parametric airfoil at angle of attack `alpha` and 
    Reynolds number `Re`, for `Nw` values (default 30 points) of the parameter `w` equally 
    spaced between 0 and 2.
    
    Return the 1D array of parameters `w`, and a 1D array with the corresponding drag coefficients. 
    """
    # define a linearly spaced array of weights (hint use np.linspace)
    ws  = np.linspace(0, 2, Nw)
    #print("This is ws", ws)
    
    # pre initialise the drag coefficient as an array of zeros of the correct size
    cds = np.zeros(Nw)
    #print("This is cds", cds)
    
    # now loop over the parameters we need to evaluate the Cd at
    for i in range(Nw):
        # YOUR CODE HERE
        # get geometry (one line)
        B, p, q = parametric_aerofoil(ws[i])
        #print(B)


        # run simulation (one line)
        cl, cd, xl, xu, cpl, cpu = airfoilsim(B, alpha, Re)
        #print(cd)

        
        # store drag coefficient at position i in cds (one line)
        cds[i] =cd
        #print(cd)

        # END YOUR CODE HERE
        
    # return the array of weights and the array of drag coefficients
    return ws, cds

### Testing

OK, to make sure your implementation is correct, run the cell below. You do not need to modify the cell, just run it.

In [17]:
# run calculations
ws, cds = parameter_sweep(alpha, Re, Nw=3)

# check
assert cds[2] == 0.0069979366

### Plotting the result

OK, if the code runs correctly, we can now plot the drag coefficient as a function of the weight $w_{\mathbf{q}_2}$. We first call our `parameter_sweep` function with default values of `alpha` and `Re` as defined in one of the cells above and then we plot the result. Complete the plotting commands are indicated.

In [18]:
# run the function with default values of angle of attack and Reynolds number 
ws, cds = parameter_sweep(alpha, Re)

In [19]:
# make a new figure
pl.figure(1, figsize=(6, 4))
pl.clf()

# BEGIN YOUR CODE HERE
# plot Cd as a function of the weight (one line)
pl.plot(ws, cds)

# add grid and labels (three lines)
pl.grid(1)
pl.ylabel("$Cd$")
pl.xlabel("$ws$")


# plot a vertical red line at the position w giving the minimum drag (one line)
pl.axvline(out.x[0], color="r")

# END YOUR CODE HERE

# as well as a green circle at the minimum
pl.plot(out.x[0], out.fun, "go")

OK, so you see the drag depends quite heavily on the airfoil geometry via the parameter `w`, and the numerical optimiser has correctly found the optimal value that minimises `Cd`. Let's move on to the next task.

---

## Task 3 - Find the airfoil with minimum drag producing a target lift coefficient $C_l^*=0.3$

Minimising drag is useful, but only if constraints on lift are met! For an aircraft flying at a given velocity and altitude (so that the air density is known), and carrying a given payload (so the total weight is known), the lift coefficient is determined. The task of the aircraft designed is to then find a geometry that minimises fuel burn!

To address this problem, we have to solve a constrained optimisation problem. We  will attempt to find the airfoil geometry (given by parameter $w_{\mathbf{q}_2}$) __and__ the angle of attack that minimise the drag coefficient whilst satifying a constraint that the lift coefficient should be equal to a target value. Simply varying $w_{\mathbf{q}_2}$ is not sufficient, because we do not have enough "flexibility" to vary `cl` and `cd` separately with only one design parameter. We could define a `parametric_airfoil_2D` function where we change the weights of two control points, but we would still have the angle of attack to take into account. 

Our design parameters are thus $w_{\mathbf{q}_2}$ __and__ `alpha`, the angle of attack, while the objective function is the drag coefficient. We could parametrise the geometry further, for instance, by allowing all weights to vary, or changing the location of the control points. This is possible but it adds complexity. At this stage, we want to understand the basic concepts.

Mathematically the optimisation problem we solve is

\begin{equation}
    \begin{aligned}
        &\underset{ w_{\mathbf{q}_2}, \alpha }{\text{min}}&& C_d{(w_{\mathbf{q}_2}, \alpha)}\\
        &\text{subject to} && C_l(w_{\mathbf{q}_2}, \alpha) = 0.3 \\
    \end{aligned}
\end{equation}

where we have a constraint on the lift coefficient. This problem is very similar in structure to the problem we solved using the penalty technique in the Python demo lecture number 4 (go watch it again if needed), and to what discussed in the lecture.

### Exploring the parameter space

We will first explore how drag and lift depend on these two parameters. We will compute the force coefficients for a range of parameters $w_{\mathbf{q}_2}$ and angles of attack. 

As shown in the Python demo lecture number 4, you should first define one array `ws` for the weight and one array `alphas` for the  angles of attack `alpha`. Fifteen points for `ws` between 0.2 and 1.5 and seventeen points for `alphas` between -0.5 and 1.0 should be used. You should then initialise two 2D arrays for the lift and draft coefficients, and name them `cls` and `cds`.

In [20]:
# Number of points for parameter w and for alpha
Nw     = 15
Nalpha = 17

# BEGIN YOUR CODE HERE
# create equally spaced point (two points)
ws  = np.linspace(0.2, 1.5, Nw)
alphas = np.linspace(-0.5, 1.0, Nalpha)

# Initialise 2D arrays for lift and drag (two lines)

cls = np.zeros(shape=(Nalpha,Nw))
cds = np.zeros(shape=(Nalpha,Nw))
#print(cls)
#print(cds)
# END YOUR CODE HERE

#### Intermediate tests
Run the tests below to check the code in the previous cell

In [21]:
assert len(ws) == 15
assert alphas[0] == -0.5
assert alphas[-1] == 1.0
assert ws[0] == 0.2
assert ws[-1] == 1.5
assert cls.shape == (17, 15)

As shown in the python demo lecture, we then fill these two arrays by using two nested for loops, to loop over all combinations of weight and angle of attack. The cell below will take a while to complete, since we are running `Nw * Nalpha` airfoil simulations! That's why a full exploration of parameter space is often prohibitevely expensive and we use optimisation techniques instead to descend to a minimum more cheaply.

In [22]:
# this will take a while to finish! we are running airfoilsim Nw*Nalpha = 15*17 times!
# Note how I use two for loops with different iteration variables for the two parameters
# You can use `iw` and `ia` to index into `ws`, `alphas` and `cls`, `cds`
for iw in range(Nw):
    for ia in range(Nalpha):
        ### BEGIN YOUR CODE HERE 
        # construct geometry (one line)
        B, p, q = parametric_aerofoil(ws[iw])

        
        # simulate at given angle of attack (one line)
        cl, cd, xl, xu, cpl, cpu = airfoilsim(B, alphas[ia], Re)
        

        
        # store values (two lines of code)
        cls[ia][iw] = cl
        cds[ia][iw] = cd

        ### END YOUR CODE HERE

Let's plot the results. I have plotted the lift coefficient contour plot in subplot 121. You should complete the cell below to also plot the drag coefficient in subplot 122 (feel free to reuse the code for subplot 121).

In [48]:
# Create figure and increase spacing between subplots
pl.figure(1, figsize=(18, 6))
pl.subplots_adjust(wspace=0.25)


# FIRST SUBPLOT - LIFT COEFFICIENT
pl.subplot(121)
# plot contours of lift coefficients versus parameter w and angle of attack alpha
pl.contourf(ws, alphas, cls, 100, cmap="Reds")
pl.colorbar(label="$C_l$")

# plot the contour for c_l = 0.3, our target c_l
c = pl.contour(ws, alphas, cls, [0.3], colors="k")

# add a label to the contour
pl.clabel(c, fmt="$C_l = %.1f$")

# add labels and a title
pl.xlabel("w")
pl.ylabel("alpha")
pl.title("Lift coefficient")


# SECOND SUBPLOT - DRAG COEFFICIENT
pl.subplot(122)

# BEGIN YOUR CODE HERE
# plot contours of drag coefficients versus parameter w and angle of attack alpha
# use `gist_earth_r` for cmap (one line)
pl.contourf(ws, alphas, cds, 100, cmap="gist_ncar_r")

# add a colorbar with label (one line)
pl.colorbar()

# plot the contour for c_l = 0.3, our target c_l and the contour label (one line)
c = pl.contour(ws, alphas, cls, [0.3], colors="k")


# add a label to the contour (one line)
pl.clabel(c, fmt="$C_l = %.1f$")

# add labels and a title
pl.xlabel("w")
pl.ylabel("alpha")
pl.title("DRAG COEFFICIENT")
# BEGIN YOUR CODE HERE

Text(0.5, 1.0, 'DRAG COEFFICIENT')

There might be a couple of white spots in these plots. These corresponds to combinations of weight and angle of attack for which the solver failed to converge and did not produce a result. These do not matter, we can still see the overall picture.

There are several interesting comments on the plots above. Looking at the lift coefficient plot on the left:

- the lift coefficient increases with `alpha` as well as with `w`
- we can vary these two parameters to keep `c_l` constant to the target 0.3, indicated by the black contour.

Looking at the drag coefficient plot on the right

- there is a combination of `w` and `alpha` that gives minimum drag (the white spot), but the lift coefficient in that region is lower than the target
- along the contour line at `c_l=0.3` (our constraint) there should be a point where the drag is minimum, at `alpha` approximately 0.4

Let's try to find the optimum point with the optimiser!

### Optimisation

As shown in lecture number 4, we try to satisfy the lift coefficient constraint using a penalty method. To do this we modify the original objective (the drag coefficient) to include a penalisation term. I have made a start in the function below, but you should complete the missing parts.

In [49]:
def drag_modified(params, cl_target=0.3):
    """
    Given the input argument `params`, a two-element array or list containing 
    the weight `w` for control point q_2 and the angle of attack `alpha`, return 
    the quantity
    
        J = cd + 10 * (cl - cl_target)**2
        
    where `cl_target=0.3` by default, and `cl` and `cd` are the lift and drag 
    coefficients of an airfoil obtained with the `parametric_airfoil` function 
    defined in Lab 2.
    """
    # unpack arguments
    w, alpha = params
    
    # obtain geometry with `parametric_airfoil`
    B, p, q = parametric_aerofoil(w)
    
    # run calculation at a given angle of attack and using the 
    # Reynolds number defined in the previous cell
    cl, cd, xl, xu, cpl, cpu = airfoilsim(B, alpha, Re)
    
    # compute objective, including constraint
    # BEGIN YOUR CODE HERE
    J = cd + 10 * (cl - cl_target)**2
    # END YOUR CODE HERE
    
    # return modified objective
    return J

### Testing

If your implementation is correct, the test below should pass

In [50]:
# run with Shift+Enter
assert drag_modified([1, 2]) == 0.541858992574313

### Optimisation

This time it's your turn to call the `minimize` function on the `drag_modified` objective function. Use a reasonable initial guess for `w` and `alpha` (see the contour plots above) and __remember to specify__ `method="Nelder-Mead"` and a suitable tolerance (not to tight to start with, e.g. `1e-2` should be OK). Store the optimisation result in a variable named `out`.

In [51]:
# BEGIN YOUR CODE HERE
guess= [0.6,0.5]
out = minimize(drag_modified, guess, tol=1e-2, method="Nelder-Mead")

# END YOUR CODE HERE

Let's print the optimisation result to see if the procedure converged successfully.

In [52]:
out

 final_simplex: (array([[0.64949375, 0.34765892],
       [0.65462837, 0.34183884],
       [0.65420094, 0.34460564]]), array([0.00527546, 0.00527676, 0.00527895]))
           fun: 0.005275459598905102
       message: 'Optimization terminated successfully.'
          nfev: 27
           nit: 12
        status: 0
       success: True
             x: array([0.64949375, 0.34765892])

Hopefully, this should be successful. Let's now plot the optimum design as a star on our parameter space.

In [53]:
# Create figure and increase spacing between subplots
pl.figure(1, figsize=(18, 12))
pl.subplots_adjust(wspace=0.25)

# plot contours of drag coefficients versus parameter w and angle of attack alpha
pl.contourf(ws, alphas, cds, 50, cmap="gist_earth_r")
pl.colorbar(label="$C_d$")

# plot the contour for c_l = 0.3, our target c_l
c = pl.contour(ws, alphas, cls, [0.3], colors="k")
pl.clabel(c, fmt="$C_l = %.1f$")

# plot the optimum design 
pl.plot(out.x[0], out.x[1], "r*", ms=20)

# add grid, labels and a title
pl.grid()
pl.xlabel("w")
pl.ylabel("alpha")
pl.title("Drag coefficient")

Text(0.5, 1.0, 'Drag coefficient')

We have found an optimum design that produces minimum drag while producing the required lift. Neat!

__End of Lab__: Well done!

In [54]:
pwd

'C:\\Users\\jonat'